# Recipe Site Traffic Prediction

#### Problem statement:
The marketing department selects a recipe each day to post on the homepage of the company's website. They have noticed that traffic to the rest of the website increases by as much as 40% if they pick a popular recipe. Since high traffic leads to more subscriptions to the company's services, they want to maximize traffic to their website.
Currently, they don't know how to determine whether a recipe will be popular, so they want a reliable solution that can select popular recipes at least 80% of the time.

## Data Validation and Data Cleaning

This data set has 947 rows and 8 columns. I have validated all variables and made the following changes:

- recipe: Numeric, unique ID, no missing values. Removed from the dataset.
- calories: Numeric, 52 missing values.
- carbohydrate: Numeric, 52 missing values.
- sugar: Numeric, 52 missing values.
- protein: Numeric, 52 missing values.
- category: Object, no missing values, 11 uniqe values. 'Chicken Breast' converted to 'Chicken'. 10 unique categories remained, just as described.
- servings: Object, no missig values. Cleaned 3 records with ' as a snack'.
- high_traffic: Object, 373 missing values representing not High traffic recipes. Column mapped as: missig values(not high traffic recipes) - 0, 'High'(high traffic recipes) - 1.

Missing values in column 'calories', 'carbohydrate', 'sugar' and 'protein' are correlated and therefore records with missing data are dropped from the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno